<a href="https://colab.research.google.com/github/Amirhossein2034/First_Script/blob/main/Stock_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import feedparser
import yfinance as yf
import requests
import time
from datetime import datetime, timedelta
from transformers import pipeline
from IPython.display import display, HTML
from collections import Counter

# Replace this with your actual NewsAPI key
NEWSAPI_KEY = "New API Key"

# Ticker to Company mapping
ticker_map = {
    "AAPL": "Apple", "MSFT": "Microsoft", "GOOGL": "Google", "AMZN": "Amazon", "META": "Meta",
    "NVDA": "Nvidia", "AMD": "AMD", "INTC": "Intel", "JPM": "JPMorgan", "BAC": "Bank of America",
    "WFC": "Wells Fargo", "C": "Citigroup", "GS": "Goldman Sachs", "MS": "Morgan Stanley",
    "XOM": "ExxonMobil", "CVX": "Chevron", "COP": "ConocoPhillips", "PSX": "Phillips 66",
    "SLB": "Schlumberger", "WMT": "Walmart", "TGT": "Target", "COST": "Costco", "HD": "Home Depot",
    "LOW": "Lowe's", "MCD": "McDonald's", "TSLA": "Tesla", "F": "Ford", "GM": "General Motors",
    "JNJ": "Johnson & Johnson", "PFE": "Pfizer", "MRK": "Merck"
}

tickers = list(ticker_map.keys())
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Get NewsAPI headlines
def fetch_newsapi(ticker):
    today = datetime.utcnow()
    yesterday = today - timedelta(days=1)
    url = (
        f"https://newsapi.org/v2/everything?q={ticker_map[ticker]}+stock&"
        f"from={yesterday.strftime('%Y-%m-%d')}&to={today.strftime('%Y-%m-%d')}&"
        f"sortBy=publishedAt&language=en&apiKey={NEWSAPI_KEY}"
    )
    try:
        res = requests.get(url)
        data = res.json()
        return [(a["title"], a["url"], a["publishedAt"][:10]) for a in data.get("articles", [])]
    except:
        return []

# Combine multiple sources
def fetch_all_news(ticker):
    company = ticker_map[ticker]
    news = set()

    # Google News RSS
    g_feed = feedparser.parse(f"https://news.google.com/rss/search?q={company}+stock&hl=en-US&gl=US&ceid=US:en")
    for entry in g_feed.entries[:5]:
        news.add((entry.title, entry.link, entry.published[:10]))

    # Yahoo Finance RSS
    y_feed = feedparser.parse(f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US")
    for entry in y_feed.entries[:5]:
        news.add((entry.title, entry.link, entry.published[:10]))

    # NewsAPI
    for item in fetch_newsapi(ticker):
        news.add(item)

    return list(news)

# Create sentiment record
def analyze_news(ticker):
    records = fetch_all_news(ticker)
    if not records:
        return [], "⚠️ No News"
    sentiments = classifier([r[0] for r in records])
    df = pd.DataFrame([{
        "Date": r[2],
        "Ticker": ticker,
        "Company": ticker_map[ticker],
        "Headline": r[0],
        "Sentiment": s["label"],
        "Confidence": round(s["score"], 3),
        "URL": r[1]
    } for r, s in zip(records, sentiments)])
    return df, generate_signal(df, ticker)

# Generate investment signal
def generate_signal(df, ticker):
    pos = sum(df["Sentiment"] == "POSITIVE")
    neg = sum(df["Sentiment"] == "NEGATIVE")
    avg_pos = df[df["Sentiment"] == "POSITIVE"]["Confidence"].mean()
    avg_neg = df[df["Sentiment"] == "NEGATIVE"]["Confidence"].mean()
    try:
        trend = yf.Ticker(ticker).history(period="5d")
        slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
    except:
        slope = 0
    if pos >= 2 and avg_pos >= 0.92 and slope > 0:
        return "✅ STRONG BUY"
    elif neg >= 2 and avg_neg >= 0.92 and slope < 0:
        return "🔻 STRONG SELL"
    elif pos > neg:
        return "🟢 BUY WATCHLIST"
    elif neg > pos:
        return "🔴 SELL WATCHLIST"
    else:
        return "🟡 HOLD"

# Process all tickers
all_news = []
signals = []

for ticker in tickers:
    try:
        news_df, signal = analyze_news(ticker)
        if not news_df.empty:
            all_news.append(news_df)
        signals.append({"Ticker": ticker, "Signal": signal})
        time.sleep(1.5)  # ✅ Prevent NewsAPI rate limit issues
    except Exception as e:
        print(f"⚠️ {ticker}: Error - {e}")
        signals.append({"Ticker": ticker, "Signal": "⚠️ Error"})

# Display Results
signal_df = pd.DataFrame(signals)
if all_news:
    combined_news = pd.concat(all_news)
    combined_news["Headline"] = combined_news.apply(lambda x: f'<a href="{x["URL"]}" target="_blank">{x["Headline"]}</a>', axis=1)
    display(HTML(combined_news[["Date", "Ticker", "Headline", "Sentiment", "Confidence"]].to_html(escape=False, index=False)))

summary = Counter(signal_df["Signal"])
print("\n📊 Signal Summary:")
for s, c in summary.items():
    print(f"{s}: {c}")

print("\n📌 Final Trading Signals:\n")
display(signal_df)


Device set to use cpu
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

⚠️ BAC: Error - URL can't contain control characters. '/rss/search?q=Bank of America+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')
⚠️ WFC: Error - URL can't contain control characters. '/rss/search?q=Wells Fargo+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')


/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0


⚠️ GS: Error - URL can't contain control characters. '/rss/search?q=Goldman Sachs+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')
⚠️ MS: Error - URL can't contain control characters. '/rss/search?q=Morgan Stanley+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')


/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]

...

⚠️ GM: Error - URL can't contain control characters. '/rss/search?q=General Motors+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')
⚠️ JNJ: Error - URL can't contain control characters. '/rss/search?q=Johnson & Johnson+stock&hl=en-US&gl=US&ceid=US:en' (found at least ' ')


/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0
/tmp/ipython-input-803776988.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slope = (trend["Close"][-1] - trend["Close"][-3]) / trend["Close"][-3] if len(trend) >= 3 else 0


Date,Ticker,Headline,Sentiment,Confidence
2025-08-04,AAPL,Tim Cook reveals shocking shift in how Americans are buying Apple products this year,POSITIVE,0.988
2025-08-04,AAPL,Apple (NASDAQ:AAPL) Price Target Raised to $255.00,NEGATIVE,0.989
2025-08-04,AAPL,"What Is Fractional Real Estate Investing, and Why Are People Flocking to It in 2025?",NEGATIVE,0.986
2025-08-04,AAPL,"Despite Billions In Investment, AI Reasoning Models Are Falling Short",NEGATIVE,1.000
2025-08-04,AAPL,"Pioneer 40” LED HD Smart TV Breaks the Three-Digit Barrier, Nearly Free Only at Best Buy",NEGATIVE,0.987
2025-08-04,AAPL,Apple Watch Ultra 2 (GPS + Cellular) 49mm Black Titanium Case $650,NEGATIVE,0.979
2025-08-04,AAPL,Dow futures drop as recession alarm bells jolt Wall Street awake from dreams of a gravity-defying economy,POSITIVE,0.986
2025-08-04,AAPL,BAB Inc. (OTCMKTS:BABB) Sees Large Growth in Short Interest,NEGATIVE,0.986
2025-08-04,AAPL,Apple’s (AAPL) “Buy” Rating Reiterated at DA Davidson,NEGATIVE,0.987
2025-08-04,AAPL,"How much does Apple pay its employees in AI, software and other roles?",NEGATIVE,0.999



📊 Signal Summary:
✅ STRONG BUY: 10
🔴 SELL WATCHLIST: 6
⚠️ Error: 8
🔻 STRONG SELL: 5
🟢 BUY WATCHLIST: 2

📌 Final Trading Signals:



,Ticker,Signal
0,AAPL,✅ STRONG BUY
1,MSFT,✅ STRONG BUY
2,GOOGL,✅ STRONG BUY
3,AMZN,✅ STRONG BUY
4,META,✅ STRONG BUY
5,NVDA,✅ STRONG BUY
6,AMD,✅ STRONG BUY
7,INTC,🔴 SELL WATCHLIST
8,JPM,🔴 SELL WATCHLIST
9,BAC,⚠️ Error
